# Abstraction and Visualization
## Coordinates to Databse to Web Interface
### Spatial Data Science Lab 1
### *Create a polygon in space, serve it to your database with flask, visualize it in ArcPro and ArcOnline with your API*


In [3]:
# import libraries
import arcpy
import psycopg2
from psycopg2 import sql

## Create Polygon from ArcGIS Primitives

In [4]:
# Create List of Points
# these are around lyn-lake area of minneapolis, points taken from google maps
input_pts= [arcpy.Point(-93.28799740050877, 44.948356410427394), arcpy.Point(-93.27799812501203, 44.946594745325356), arcpy.Point(-93.27791229432108, 44.93775523014747), arcpy.Point(-93.2984043787839, 44.937687788845565), arcpy.Point(-93.28799740050877, 44.948356410427394)]
# Convert to an Array then to a Polygon
my_polygon = arcpy.Polygon(arcpy.Array(input_pts))

## Push the table up to my PostGIS database with psycopg and SQL

In [9]:
# use psycopg to connect to my database hosted on Google Cloud 
connection = psycopg2.connect(host = 'your_ip', #ip address #all these values must be strings!
                             database = 'lab0', #name of database created in your sql instance
                             user = 'postgres',
                             password = 'your_pw'##insert password, do not leave it here when post to github!
                             )

#instantiate a cursor, make the SQL table from the poly variable above
cursor = connection.cursor()
my_table = sql.SQL("CREATE TABLE IF NOT EXISTS new_polygon (polygon_id SERIAL PRIMARY KEY, geometry GEOMETRY)")
cursor.execute(my_table)

# format of the above cursor.execute("SQL QUERY name of table (name of first column SQL DATA TYPE, name of second column SQL DATATYPE)")
# the above uses SQL queries to create a table named new_polygon and name it's columns polygon_id and geometry
# the datatypes are set as Serial Primary Key (giving a serial index of unique numbers in that column)
# and Gemoetry which accepts tuples of x, y pairings

#insert values from the arcpy polygon defined above and
#convert values in the "geometry" column to well know text format using the insert SQL query
cursor.execute("INSERT INTO new_polygon (geometry) VALUES (ST_GeomFromText('{}'))".format(my_polygon.WKT))
# uses the postGIS ST_GeomFromText() to create geometry objects from the geometry
#column from new_polygon table object tuples with no CRS then formats the whole table
#as well known text

#set the crs/spatial reference system of the geometry column to web mercatur 
cursor.execute("SELECT ST_SetSRID(geometry, 4326) FROM new_polygon;")

#end the cursor instance
cursor.close



<built-in method close of psycopg2.extensions.cursor object at 0x0000025C51695660>

In [10]:
# test the connection, a valid connection will return 0
connection.closed

0

In [48]:
#look at the first cell of table I've just created

# create a cursor again 
cursor = connection.cursor()

# use the cursor to select the first cell of the table with the SQL command select 1 and return it with fetchall
cursor.execute("SELECT 1")
cursor.fetchall()

InterfaceError: connection already closed

In [14]:
#safely save the changes and end connection
connection.commit()
cursor.close() #end the cursor instance
connection.close()

InterfaceError: connection already closed

## Create a Flask App to allow the polygon to be downloaded through an API for the database

In [ ]:
#Set up the App
#Import libraries and Flask
#from flask import Flask, jsonify #do not import flask to the arc environment!
import psycopg2

#Instantiate flask,
#app = Flask(__name__)

In [ ]:
# Connect to the PostgreSQL database

#Tip from Rob, set credentials as dictionary
#then use the conn variable to unpack the dict
#with equals signs using the two stars method
pg_connect_dict = {
    'dbname' : 'lab0',
    'user' : 'postgres',
    'password' : 'your_pw',
    'host' : 'your_ip'
}

conn = psycopg2.connect(**pg_connect_dict) #unpack the dict above in the correct format

#define the decorator functions

@app.route('/') #python decorator
def hello_world(): #function that app.route decorator references
  response = hello()
  return response

def hello():
  return "hello, world"

#define another decorator
@app.route('/geojson', methods=['POST', 'GET'])
def get_geojson():
    # Execute a query to retrieve the polygon from the database
    cursor = conn.cursor()
    cursor.execute("SELECT ST_AsGeoJSON(new_polygon.*) FROM new_polygon;")
    result = cursor.fetchall()
    return result[0][0]
    
    if result is None:
        return jsonify({'error': 'Polygon not found'}), 404
    else:
        return jsonify({'geojson': result[0]})
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)

@app.route('/polygon/<polygon_id>', methods=['POST', 'GET'])
#define another page decorator
def get_polygon(polygon_id):
    
    cursor = conn.cursor()
    cursor.execute("SELECT ST_AsGeoJSON(geometry) FROM new_polygon WHERE polygon_id = %s;", (polygon_id,))
    result = cursor.fetchone()

 # Return the result as a GeoJSON object
    if result is None:
        return jsonify({'error': 'Polygon not found'}), 404
    else:
        return jsonify({'geojson': result[0]})
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)


# get the geojson through the api

In [ ]:
# Retrieve polygon from VM API
# url = r'http://34.28.9.238:5000/geojson'
# dictionary = json.loads(requests.get(url).text)

# # Remove one pair of brackets  
# sub = dictionary['geojson'].replace("[[[[", '[[[').replace(']]]]', ']]]')

# # Convert string to JSON dictionary
# dictionary_2 = json.loads(sub)

## Import the Geojson with a GET request

In [1]:
import requests
import json

In [32]:
# set working environment
cwd = r'C:\Users\MrJDF\OneDrive\Documents\ArcGIS\Projects\DatabaseConnectionLab0'

# set url of my api with the geojson endpoint from the flask app
url = r'http://34.28.9.238:5000/geojson'

#make request
response = requests.get(url)    

GEoJSON saved!


In [ ]:
#unpack request as json

with open (cwd + '\my_new_GeoJSON.geojson', 'w') as file:
    file.write(str(response.json()))
    print('GEoJSON saved!')

In [50]:
# convert the geojson into a feature class

geojson_in = cwd + '\my_new_GeoJSON.geojson'

arcpy.conversion.JSONToFeatures(geojson_in, 'LynLakePolygon')

<Result 'C:\\Users\\MrJDF\\OneDrive\\Documents\\ArcGIS\\Projects\\DatabaseConnectionLab0\\DatabaseConnectionLab0.gdb\\LynLakePolygon'>

In [47]:
# add layer from path - this path is with the flask app
#runnign in google cloud run,
#after building with google build

# this doesn't work BC esri won't accept geojson, so have to do the conversion in the cells above

# Executing tool will automatically add layer to map
# arcpy.management.MakeFeatureLayer(r'https://cloud-run-test-6omxj2spba-uc.a.run.app/geojson', "LynLakeAPILayer") 